In [3]:
!gpustat -cu

n104.mcl.weizmann.ac.il  Wed Jan 20 19:22:16 2021  450.80.02
[0] GeForce RTX 2080 Ti | 27'C,   0 % |     0 / 11019 MB |
[1] GeForce RTX 2080 Ti | 29'C,   0 % |  2508 / 11019 MB | benf:python(2505M)
[2] GeForce RTX 2080 Ti | 25'C,   0 % |     0 / 11019 MB |
[3] GeForce RTX 2080 Ti | 25'C,   0 % |     0 / 11019 MB |
[4] GeForce RTX 2080 Ti | 26'C,   0 % |     0 / 11019 MB |
[5] GeForce RTX 2080 Ti | 26'C,   0 % |     0 / 11019 MB |
[6] GeForce RTX 2080 Ti | 25'C,   0 % |     0 / 11019 MB |
[7] GeForce RTX 2080 Ti | 24'C,   0 % |     0 / 11019 MB |


In [4]:
%env CUDA_VISIBLE_DEVICES 0

env: CUDA_VISIBLE_DEVICES=0


In [5]:
import itertools
import math
import os
import pickle
import random
import time
from pprint import pprint

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

import Resizer
from image import imread, imwrite, imshow, to_numpy

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', DEVICE)

device: cuda


In [6]:
def np2pt(arr):
    return torch.tensor(arr).permute(2, 0, 1).unsqueeze(0).contiguous()

def pt2np(tensor):
    return to_numpy(tensor.squeeze(0).permute(1, 2, 0))

def bm3d_denoise(input, sigma):
    return np2pt(bm3d(pt2np(input), sigma)).to(device=input.device, dtype=input.dtype)

def imread2pt(path, d=None, **kwargs):
    image = imread(path, **kwargs)
    image = np2pt(image)
    # image = torch.tensor(imread(path, **kwargs)).permute(2, 0, 1).unsqueeze(0)
    if d is not None:
        image = shave_image(image, d)
    return image.contiguous()


In [11]:
def get_pyramids_aux(img_torch, num_layers,ratio, print_):
    max_layer = num_layers
    pyr = []
    ratio = ratio
    if isinstance(ratio, float) or isinstance(ratio, int):
        ratio = (ratio, ratio)
    curr_img = img_torch
    pyr.append(curr_img)
    for j in range(max_layer):
        if print_:
            print(curr_img.shape)
            imshow(curr_img)
        rsizer = Resizer.Resizer(curr_img.shape, ratio, [1, img_torch.shape[1], np.ceil(img_torch.shape[2]*(ratio[0])**(j+1)), np.ceil(img_torch.shape[3]*(ratio[1])**(j+1))]).cuda()
        curr_img = rsizer(curr_img)
        pyr.append(curr_img)
        
    return pyr

def get_pyramid(file, degradation, sigma, num_layers, ratio, print_=True):
    gt = np.expand_dims(imread(file, pt=False), 0)
    gt = torch.tensor(gt).contiguous().permute(0, 3, 1, 2).detach().cuda()
    return get_pyramids_aux(gt, num_layers,ratio, print_)

In [8]:
DIV = 100

def patch2im(input, w, h, patch_size=7):
    out = input.transpose(1,2)
    normalize = torch.ones_like(out)
    normalize = normalize.transpose(1,2)
    fold = torch.nn.functional.fold(
        input, 
        output_size=(w, h), 
        kernel_size=(patch_size, patch_size)
    )

    norm = torch.nn.functional.fold(
        normalize,
        output_size=(w, h),
        kernel_size=(patch_size, patch_size),
    )

    return fold/norm


def _calc_dist_l2(X, Y):
    Y = Y.transpose(0, 1)
    X2 = X.pow(2).sum(1, keepdim=True)
    Y2 = Y.pow(2).sum(0, keepdim=True)
    XY = X @ Y
    return X2 - (2 * XY) + Y2


def build_image(input_img, index_imgs, ref_imgs, patch_size=7):
    unfold = torch.nn.Unfold(kernel_size=(patch_size,patch_size))
    in_patches = unfold(input_img)
    for i, (ind_img, ref_img) in enumerate(zip(index_imgs, ref_imgs)):
        if i == 0:
            index_patches = unfold(ind_img)
            ref_patches = unfold(ref_img)
        else:
            index_patches = torch.cat([index_patches, unfold(ind_img)],dim=-1)
            ref_patches = torch.cat([ref_patches, unfold(ref_img)], dim=-1)
            
    if in_patches.shape[-1]*ref_patches.shape[-1] > 14000**2:
        for j in range(DIV):
            start_patch = j*(math.ceil(in_patches.shape[-1]/DIV))
            end_patch = min((j+1)*(math.ceil(in_patches.shape[-1]/DIV)), in_patches.shape[-1])
            dist_mat = _calc_dist_l2(in_patches[:,:,start_patch:end_patch].permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
#                 print(start_patch, end_patch, dist_mat.shape)
            if j == 0:
                ind = dist_mat.argmin(1)
            elif start_patch < end_patch:
                ind = torch.cat([ind, dist_mat.argmin(1)])

    else:
        dist_mat = _calc_dist_l2(in_patches.permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
        ind = dist_mat.argmin(1)
    out_patches = F.embedding(ind, ref_patches.squeeze(0).permute(1,0))
    return patch2im(out_patches.unsqueeze(0).permute(0,2,1), input_img.shape[-2], input_img.shape[-1], patch_size)

In [9]:
def new_image_generation(src_pyr, dst_pyr, patch_size=7, top_level=9):
    start = time.time()
    new_im = dst_pyr[top_level] + torch.randn_like(dst_pyr[top_level])*0.75#new_pyr[9].clone()#
    for l in range(top_level,-1,-1): 
        num_iters = 1 if l == top_level else 10
        for k in range(num_iters):
            rsizer = Resizer.Resizer(src_pyr[l+1].shape, 4/3, src_pyr[l].shape).cuda()
            if l == top_level:
                new_im = build_image(new_im, [src_pyr[l]] ,[src_pyr[l]], patch_size=7)
            else:
                new_im = build_image(new_im, [rsizer(src_pyr[l+1])] ,[src_pyr[l]], patch_size=7)
        if l > 0:
            rsizer = Resizer.Resizer(dst_pyr[l].shape, 4/3, dst_pyr[l-1].shape).cuda()
            new_im = rsizer(new_im)
    imshow(new_im)
    print('Total time: %.2f[s]' % (time.time() - start,))
    return new_im

In [14]:
pyr = get_pyramid('/home/nivg/data/mountains.jpg', None, 0, 15, 0.75, print_=False) 
pyr=pyr[3:]
for ratio in ([[1,1],[1,4/5],[1,3/4],[1,1/3],[1,5/4],[1,2]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        start = time.time()
        out = new_image_generation(pyr, dst_pyr)
        print('ratio=%s' % (ratio,), 'k=%d' % (k,), 'time=%.2fms' % (1000 * (time.time() - start),))
        imshow(out)
        
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xf7\x00\x00\x00\x9b\x08\x02\x00\x00\x001\xd8\x9a…

Total time: 1.99[s]
ratio=[1, 1] k=0 time=1992.02ms


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xf7\x00\x00\x00\x9b\x08\x02\x00\x00\x001\xd8\x9a…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xf7\x00\x00\x00\x9b\x08\x02\x00\x00\x001\xd8\x9a…

Total time: 1.98[s]
ratio=[1, 1] k=1 time=1978.65ms


...

Total time: 3.50[s]
ratio=[1, 2] k=0 time=3498.42ms


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xee\x00\x00\x00\x9b\x08\x02\x00\x00\x00+\xed\xef…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xee\x00\x00\x00\x9b\x08\x02\x00\x00\x00+\xed\xef…

Total time: 3.50[s]
ratio=[1, 2] k=1 time=3499.69ms


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xee\x00\x00\x00\x9b\x08\x02\x00\x00\x00+\xed\xef…

In [ ]:
DIV = 100

def patch2im(input, w, h, patch_size=7):
    out = input.transpose(1,2)
    normalize = torch.ones_like(out)
    normalize = normalize.transpose(1,2)
    fold = torch.nn.functional.fold(
        input, 
        output_size=(w, h), 
        kernel_size=(patch_size, patch_size)
    )

    norm = torch.nn.functional.fold(
        normalize,
        output_size=(w, h),
        kernel_size=(patch_size, patch_size),
    )

    return fold/norm


def _calc_dist_l2(X, Y):
    Y = Y.transpose(0, 1)
    X2 = X.pow(2).sum(1, keepdim=True)
    Y2 = Y.pow(2).sum(0, keepdim=True)
    XY = X @ Y
    return X2 - (2 * XY) + Y2


def build_image(input_img, index_imgs, ref_imgs, patch_size=7):
    unfold = torch.nn.Unfold(kernel_size=(patch_size,patch_size))
    in_patches = unfold(input_img)
    for i, (ind_img, ref_img) in enumerate(zip(index_imgs, ref_imgs)):
        if i == 0:
            index_patches = unfold(ind_img)
            ref_patches = unfold(ref_img)
        else:
            index_patches = torch.cat([index_patches, unfold(ind_img)],dim=-1)
            ref_patches = torch.cat([ref_patches, unfold(ref_img)], dim=-1)
            
    if in_patches.shape[-1]*ref_patches.shape[-1] > 14000**2:
        for j in range(DIV):
            start_patch = j*(math.ceil(in_patches.shape[-1]/DIV))
            end_patch = min((j+1)*(math.ceil(in_patches.shape[-1]/DIV)), in_patches.shape[-1])
            dist_mat = _calc_dist_l2(in_patches[:,:,start_patch:end_patch].permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
#                 print(start_patch, end_patch, dist_mat.shape)
            if j == 0:
                ind = dist_mat.argmin(1)
            elif start_patch < end_patch:
                ind = torch.cat([ind, dist_mat.argmin(1)])

    else:
        dist_mat = _calc_dist_l2(in_patches.permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
        ind = dist_mat.argmin(1)
    out_patches = F.embedding(ind, ref_patches.squeeze(0).permute(1,0))
    return patch2im(out_patches.unsqueeze(0).permute(0,2,1), input_img.shape[-2], input_img.shape[-1], patch_size)

def build_image_bidirectional(input_img, index_imgs, ref_imgs, patch_size=7, eps=1):
    unfold = torch.nn.Unfold(kernel_size=(patch_size,patch_size))
    in_patches = unfold(input_img)
    for i, (ind_img, ref_img) in enumerate(zip(index_imgs, ref_imgs)):
        if i == 0:
            index_patches = unfold(ind_img)
            ref_patches = unfold(ref_img)
        else:
            index_patches = torch.cat([index_patches, unfold(ind_img)],dim=-1)
            ref_patches = torch.cat([ref_patches, unfold(ref_img)], dim=-1)
            
    if in_patches.shape[-1]*ref_patches.shape[-1] > 14000**2:
        for j in range(DIV):
            start_patch = j*(math.ceil(in_patches.shape[-1]/DIV))
            end_patch = min((j+1)*(math.ceil(in_patches.shape[-1]/DIV)), in_patches.shape[-1])
            dist_mat = _calc_dist_l2(in_patches[:,:,start_patch:end_patch].permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
            dist_mat /= patch_size*patch_size*3
            if start_patch < end_patch:
                dist_mat = dist_mat/(dist_mat.min(0, keepdim=True)[0] + eps)
            if j == 0:
                ind = dist_mat.argmin(1)
            elif start_patch < end_patch:
                ind = torch.cat([ind, dist_mat.argmin(1)])

    else:
        dist_mat = _calc_dist_l2(in_patches.permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
        dist_mat /= patch_size*patch_size*3
        dist_mat = dist_mat/(dist_mat.min(0, keepdim=True)[0] + eps)
        ind = dist_mat.argmin(1)
    out_patches = F.embedding(ind, ref_patches.squeeze(0).permute(1,0))
    return patch2im(out_patches.unsqueeze(0).permute(0,2,1), input_img.shape[-2], input_img.shape[-1], patch_size)



def build_image_no_dc(input_img, index_imgs, ref_imgs, patch_size=7):
    unfold = torch.nn.Unfold(kernel_size=(patch_size,patch_size))
    in_patches = unfold(input_img)
    for i, (ind_img, ref_img) in enumerate(zip(index_imgs, ref_imgs)):
        if i == 0:
            index_patches = unfold(ind_img)
            ref_patches = unfold(ref_img)
        else:
            index_patches = torch.cat([index_patches, unfold(ind_img)],dim=-1)
            ref_patches = torch.cat([ref_patches, unfold(ref_img)], dim=-1)
    index_patches = index_patches - index_patches.mean(1, keepdim=True)
    ref_patches = ref_patches - ref_patches.mean(1, keepdim=True)
    dc_in = in_patches.mean(1, keepdim=True)
    in_patches = in_patches - dc_in
    if in_patches.shape[-1]*ref_patches.shape[-1] > 14000**2:
        for j in range(DIV):
            start_patch = j*(math.ceil(in_patches.shape[-1]/DIV))
            end_patch = min((j+1)*(math.ceil(in_patches.shape[-1]/DIV)), in_patches.shape[-1])
            dist_mat = _calc_dist_l2(in_patches[:,:,start_patch:end_patch].permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
#                 print(start_patch, end_patch, dist_mat.shape)
            if j == 0:
                ind = dist_mat.argmin(1)
            elif start_patch < end_patch:
                ind = torch.cat([ind, dist_mat.argmin(1)])

    else:
        dist_mat = _calc_dist_l2(in_patches.permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
        ind = dist_mat.argmin(1)
    out_patches = F.embedding(ind, ref_patches.squeeze(0).permute(1,0))
    return patch2im(out_patches.unsqueeze(0).permute(0,2,1) + dc_in, input_img.shape[-2], input_img.shape[-1], patch_size)



def build_image_mask(input_img, index_imgs, ref_imgs, mask_imgs, patch_size=7):
    unfold = torch.nn.Unfold(kernel_size=(patch_size,patch_size))
    in_patches = unfold(input_img)
    for i, (ind_img, ref_img, mask_img) in enumerate(zip(index_imgs, ref_imgs, mask_imgs)):
        if i == 0:
            index_patches = unfold(ind_img)
            ref_patches = unfold(ref_img)
            mask_patches = unfold(mask_img)
        else:
            index_patches = torch.cat([index_patches, unfold(ind_img)],dim=-1)
            ref_patches = torch.cat([ref_patches, unfold(ref_img)], dim=-1)
            mask_patches = torch.cat([mask_patches, unfold(mask_img)], dim=-1)
    mask_patches = (mask_patches.mean(1).squeeze(0) == 1).byte()
    print(mask_patches.shape, ref_patches.shape)
    index_patches = index_patches[:,:,mask_patches]
    ref_patches = ref_patches[:,:,mask_patches]
    if in_patches.shape[-1]*ref_patches.shape[-1] > 14000**2:
        for j in range(DIV):
            start_patch = j*(math.ceil(in_patches.shape[-1]/DIV))
            end_patch = min((j+1)*(math.ceil(in_patches.shape[-1]/DIV)), in_patches.shape[-1])
            dist_mat = _calc_dist_l2(in_patches[:,:,start_patch:end_patch].permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
#                 print(start_patch, end_patch, dist_mat.shape)
            if j == 0:
                ind = dist_mat.argmin(1)
            elif start_patch < end_patch:
                ind = torch.cat([ind, dist_mat.argmin(1)])

    else:
        dist_mat = _calc_dist_l2(in_patches.permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
        ind = dist_mat.argmin(1)
    out_patches = F.embedding(ind, ref_patches.squeeze(0).permute(1,0))
    return patch2im(out_patches.unsqueeze(0).permute(0,2,1), input_img.shape[-2], input_img.shape[-1], patch_size)


from fold import unfold2d, fold2d, view_as_column, view_as_image
def build_image_bidirectional_weighted(input_img, index_imgs, ref_imgs, patch_size=7, eps=1):
    unfold = torch.nn.Unfold(kernel_size=(patch_size,patch_size))
    in_patches = unfold2d(input_img, use_padding=False, use_view=True)
    in_patches, patches_shape = view_as_column(in_patches)
    for i, (ind_img, ref_img) in enumerate(zip(index_imgs, ref_imgs)):
        if i == 0:
            index_patches = unfold2d(ind_img, use_padding=False)
            ref_patches = unfold2d(ref_img, use_padding=False)
        else:
            index_patches = torch.cat([index_patches, unfold2d(ind_img, use_padding=False)],dim=-1)
            ref_patches = torch.cat([ref_patches, unfold2d(ref_img, use_padding=False)], dim=-1)
            
    if in_patches.shape[-1]*ref_patches.shape[-1] > 14000**2:
        for j in range(DIV):
            start_patch = j*(math.ceil(in_patches.shape[-1]/DIV))
            end_patch = min((j+1)*(math.ceil(in_patches.shape[-1]/DIV)), in_patches.shape[-1])
            dist_mat = _calc_dist_l2(in_patches[:,:,start_patch:end_patch].permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
            dist_mat /= patch_size*patch_size*3
            if start_patch < end_patch:
                dist_mat = dist_mat/(dist_mat.min(0, keepdim=True)[0] + eps)
            if j == 0:
                ind = dist_mat.argmin(1)
            elif start_patch < end_patch:
                ind = torch.cat([ind, dist_mat.argmin(1)])

    else:
        dist_mat = _calc_dist_l2(in_patches.permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
        dist_mat /= patch_size*patch_size*3
        dist_mat = dist_mat/(dist_mat.min(0, keepdim=True)[0] + eps)
        ind = dist_mat.argmin(1)
    out_patches = F.embedding(ind, ref_patches.squeeze(0).permute(1,0)).unsqueeze(0).transpose(1,2)
    out_patches = view_as_image(out_patches, patches_shape)
    return fold2d(out_patches, reduce='weighted_mean', use_padding=False)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
        torch.nn.init.xavier_normal_(m.weight, nn.init.calculate_gain('leaky_relu'))


class Generator(nn.Module):

    def __init__(self, n_layers=8, chan=64, kernel_size=3, in_ch=3, out_ch=3, bias=True, padding_mode='reflect'):
        super(Generator, self).__init__()
        
        first_layer = [nn.Conv2d(in_channels=in_ch, out_channels=chan, kernel_size=kernel_size, bias=bias, padding = kernel_size//2, padding_mode=padding_mode),
                            nn.ReLU(True)]
        self.first_layer = nn.Sequential(*first_layer)
        feature_block = []
        for _ in range(1, n_layers - 1):
            feature_block += [nn.Conv2d(in_channels=chan, out_channels=chan, kernel_size=kernel_size, bias=bias, padding = kernel_size//2, padding_mode=padding_mode),
                              nn.ReLU(True)]
        self.feature_block = nn.Sequential(*feature_block)
        self.final_layer = nn.Sequential(nn.Conv2d(in_channels=chan, out_channels=out_ch, kernel_size=kernel_size, bias=bias, padding = kernel_size//2, padding_mode=padding_mode))


    def forward(self, input_tensor):
        receptive_extraction = self.first_layer(input_tensor)
        features = self.feature_block(receptive_extraction)
        return self.final_layer(features)



In [ ]:
# Image Generation

def new_image_generation(src_pyr, dst_pyr, patch_size=7, top_level=9):
    start = time.time()
    new_im = dst_pyr[top_level] + torch.randn_like(dst_pyr[top_level])*0.75#new_pyr[9].clone()#
    for l in range(top_level,-1,-1): 
        num_iters = 1 if l == top_level else 10
        for k in range(num_iters):
            rsizer = Resizer.Resizer(src_pyr[l+1].shape, 4/3, src_pyr[l].shape).cuda()
            if l == top_level:
                new_im = build_image(new_im, [src_pyr[l]] ,[src_pyr[l]], patch_size=7)
            else:
                new_im = build_image(new_im, [rsizer(src_pyr[l+1])] ,[src_pyr[l]], patch_size=7)
        if l > 0:
            rsizer = Resizer.Resizer(dst_pyr[l].shape, 4/3, dst_pyr[l-1].shape).cuda()
            new_im = rsizer(new_im)
    imshow(new_im)
    print('Total time: %.2f[s]' % (time.time() - start,))
    return new_im


def harmonization(src_pyr, dst_pyr, patch_size=7, top_level=1):
    start = time.time()
    new_im = dst_pyr[top_level] #new_pyr[9].clone()#
    for l in range(top_level,-1,-1): 
        num_iters = 1
        for k in range(num_iters):
            rsizer = Resizer.Resizer(src_pyr[l+1].shape, 4/3, src_pyr[l].shape).cuda()
            if l == top_level:
                new_im = build_image(new_im, [src_pyr[l]] ,[src_pyr[l]], patch_size=patch_size)
            else:
                new_im = build_image(new_im, [rsizer(src_pyr[l+1])] ,[src_pyr[l]], patch_size=patch_size)
        if l > 0:
            rsizer = Resizer.Resizer(dst_pyr[l].shape, 4/3, dst_pyr[l-1].shape).cuda()
            new_im = rsizer(new_im)
    imshow(new_im)
    print('Total time: %.2f[s]' % (time.time() - start,))
    return new_im

def edit(src_pyr, dst_pyr, mask_size, patch_size=7, top_level_max=9):
    start = time.time()
    top_level = int(np.floor(np.log(patch_size/mask_size)/np.log(3/4)))
    top_level = min(top_level_max, top_level)
    new_im = dst_pyr[top_level]
    for l in range(top_level,-1,-1): 
        num_iters = 1 if l == top_level else 10
        new_im = new_im + torch.randn_like(new_im)*(0.4**(10 -l))
        for k in range(num_iters):
            rsizer = Resizer.Resizer(src_pyr[l+1].shape, 4/3, src_pyr[l].shape).cuda()
            if l == top_level:
                new_im = build_image(new_im, [src_pyr[l]] ,[src_pyr[l]], patch_size=7)
            else:
                new_im = build_image(new_im, [rsizer(src_pyr[l+1])] ,[src_pyr[l]], patch_size=7)
        if l > 0:
            rsizer = Resizer.Resizer(dst_pyr[l].shape, 4/3, dst_pyr[l-1].shape).cuda()
            new_im = rsizer(new_im)
    imshow(new_im)
    print('Total time: %.2f[s]' % (time.time() - start,))
    return new_im


def summarization(pyr, ratio=(1/3,1/3), print_=False, patch_size=7, min_size=28, weighted_mean=False):
    start = time.time()
    new_im = pyr[0].clone()
    imshow(new_im)
    
    
    if ratio[0] <= 1 and ratio[1] <= 1:
        num_steps = math.floor(max(math.log(ratio[0])/math.log(0.9), math.log(ratio[1])/math.log(0.9)))
        step_size = [0.9, 0.9]
        if ratio[0] > ratio[1]:
            step_size[0] = 10**(math.log10(ratio[0])/num_steps)
        elif ratio[1] > ratio[0]:
            step_size[1] = 10**(math.log10(ratio[1])/num_steps)

    elif ratio[0] >= 1 and ratio[1] >= 1:
        num_steps = math.floor(max(math.log(ratio[0])/math.log(1.2), math.log(ratio[1])/math.log(1.2)))
        step_size = [1.2, 1.2]
        if ratio[0] < ratio[1]:
            step_size[0] = 10**(math.log10(ratio[0])/num_steps)
        elif ratio[1] < ratio[0]:
            step_size[1] = 10**(math.log10(ratio[1])/num_steps)
    
    print(step_size, num_steps)
    
    for i in range(num_steps):
        rsizer = Resizer.Resizer(new_im.shape, step_size).cuda()
        new_im = rsizer(new_im)
        new_pyr = get_pyramids_aux(new_im, 15, 0.8, False)

        top_level = math.floor(min(math.log(min_size/new_im.shape[-2])/math.log(0.8), math.log(min_size/new_im.shape[-1])/math.log(0.8)))
        
        print(top_level, new_pyr[top_level].shape)
        
        new_im = new_pyr[top_level].clone()#
        start = time.time()
        eps = (5e-3)
        for l in range(top_level,-1,-1): 
            num_iters = 10
            for k in range(num_iters):
                rsizer = Resizer.Resizer(pyr[l+1].shape, 5/4, pyr[l].shape).cuda()
                if weighted_mean:
                    new_im = build_image_bidirectional_weighted(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], patch_size=7, eps=eps)
                else:
                    new_im = build_image_bidirectional(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], patch_size=7, eps=eps)
            imshow(new_im)
            if l > 0:
                rsizer = Resizer.Resizer(new_pyr[l].shape, 5/4, new_pyr[l-1].shape).cuda()
                new_im = rsizer(new_im)
        print('Scale ' + str(i) + ', Total time: %.2f[s]' % (time.time() - start,))
        imshow(new_im)

        
def Montage(pyrs, num_steps=10, print_=False, patch_size=7, min_size=28):
    start = time.time()
    for i, pyr in enumerate(pyrs):
        if i == 0:
            new_im = pyr[0]
        else:
            new_im = torch.cat([new_im, pyr[0]], dim=-1)
    imshow(new_im)
    
    step_size = [1, 0.9]
    
    for i in range(num_steps):
        rsizer = Resizer.Resizer(new_im.shape, step_size).cuda()
        new_im = rsizer(new_im)
        new_pyr = get_pyramids_aux(new_im, 15, 0.8, False)

        top_level = math.floor(min(math.log(min_size/new_im.shape[-2])/math.log(0.8), math.log(min_size/new_im.shape[-1])/math.log(0.8)))
        
        print(top_level, new_pyr[top_level].shape)
        
        new_im = new_pyr[top_level].clone()#
        start = time.time()
        eps = (5e-4)
        for l in range(top_level,-1,-1): 
            num_iters = 10
            for k in range(num_iters):
                keys = []
                values = []
                for pyr in pyrs:
                    rsizer = Resizer.Resizer(pyr[l+1].shape, 5/4, pyr[l].shape).cuda()
                    keys.append(rsizer(pyr[l+1]))
                    keys.append(pyr[l])
                    values.append(pyr[l])
                    values.append(pyr[l])
                new_im = build_image_bidirectional(new_im, keys ,values, patch_size=7, eps=eps)
            if l > 0:
                rsizer = Resizer.Resizer(new_pyr[l].shape, 5/4, new_pyr[l-1].shape).cuda()
                new_im = rsizer(new_im)
        print('Shape: ' + str(new_im.shape) + ', Scale: ' + str(i) + ', Total time: %.2f[s]' % (time.time() - start,))
        imshow(new_im)

        
def structural_analogy(path_a, path_b, patch_size=7, top_layer=8, eps=5e-3, print_=False):
    start = time.time()
    pyr = get_pyramid(path_a,  None, 0, 20, 0.8, print_=print_) # Summary
    pyr2 = get_pyramid(path_b, None, 0, 20, 0.8, print_=print_) # Summary
    new_im = pyr2[0].clone()
    for i in range(2):
        new_pyr = get_pyramids_aux(new_im, 15, 0.8, print_)
        new_im = new_pyr[top_layer].clone()
        for l in range(top_layer,-1,-1): 
            num_iters = 10
            for k in range(num_iters):
                rsizer = Resizer.Resizer(pyr[l+1].shape, 5/4, pyr[l].shape).cuda()
                new_im = build_image_bidirectional(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], patch_size=7, eps=eps)
            if l > 0:
                rsizer = Resizer.Resizer(new_pyr[l].shape, 5/4, new_pyr[l-1].shape).cuda()
                new_im = rsizer(new_im)
    print('Total time: %.2f[s]' % (time.time() - start,))
    imshow(new_im)
    imshow(pyr2[0])
    return new_im


def structural_analogy_refinement(path_a, im, patch_size=7, top_layer=5, eps=5e-3, print_=False):
    start = time.time()
    pyr = get_pyramid(path_a,  None, 0, 20, 0.8, print_=print_)
    new_pyr = get_pyramids_aux(im, 15, 0.8, print_)
    new_im = new_pyr[top_layer].clone()
    for l in range(top_layer,-1,-1): 
        num_iters = 10
        for k in range(num_iters):
            rsizer = Resizer.Resizer(pyr[l+1].shape, 5/4, pyr[l].shape).cuda()
            new_im = build_image(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], patch_size=7)
        if l > 0:
            rsizer = Resizer.Resizer(new_pyr[l].shape, 5/4, new_pyr[l-1].shape).cuda()
            new_im = rsizer(new_im)
    print('Total time: %.2f[s]' % (time.time() - start,))
    imshow(new_im)
    imshow(pyr[0])
    return new_im


def conditional_inpainting(im, mask, patch_size=7, top_layer=8, print_=False):
    start = time.time()
    pyr = get_pyramids_aux(im, 15, 0.75, print_=print_) # Summary
    mask_pyr = get_pyramids_aux(mask, 15, 0.75, print_=print_) # Summary
    new_im = pyr[top_layer]
    for l in range(top_layer,-1,-1): 
        num_iters = 10
        for k in range(num_iters):
            rsizer = Resizer.Resizer(pyr[l+1].shape, 4/3, pyr[l].shape).cuda()
            new_im = build_image_mask(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], [rsizer(mask_pyr[l+1]), mask_pyr[l]], patch_size=7)
        imshow(new_im)
        if l > 0:
            rsizer = Resizer.Resizer(pyr[l].shape, 4/3, pyr[l-1].shape).cuda()
            new_im = rsizer(new_im)

    print('Total time: %.2f[s]' % (time.time() - start,))
    imshow(new_im)
    imshow(pyr[0])
    return new_im

In [ ]:
# Birds, mountains Comparison
pyr = get_pyramid('/home/nivg/data/mountains.jpg', None, 0, 15, 0.75, print_=True) 
pyr=pyr[3:]
for ratio in ([[1,1],[1,4/5],[1,3/4],[1,1/3],[1,5/4],[1,2]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        im = new_image_generation(pyr, dst_pyr)
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Mountains3 Comparison
for ratio in ([[1,1],[1,1/3],[1,5/4],[4/5,1],[1,2]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        im = new_image_generation(pyr, dst_pyr)
        imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Balloons Comparison
for ratio in ([[1,1],[1,1/3],[1,5/4],[5/4,1],[1,2]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        im = new_image_generation(pyr, dst_pyr)
        imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Balloons/Colusseum for figure
pyr = get_pyramid('/home/nivg/data/birds.png', None, 0, 15, 0.75, print_=True) 

for ratio in ([[1,1]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(10):
        im = new_image_generation(pyr, dst_pyr)
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# colusseum Comparison
pyr = get_pyramid('/home/nivg/data/colusseum.png', None, 0, 15, 0.75, print_=True) 
# pyr=pyr[3:]
imshow(pyr[0])
for ratio in ([[1,1],[1,4/3],[4/3,1]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        if ratio == [4/3,1]:
            im = new_image_generation(pyr, dst_pyr, top_level=8)
        else:
            im = new_image_generation(pyr, dst_pyr)
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Seascape Harmonization
IM_PATH = '/home/nivg/data/seascape.png'
HARM_IM_PATH = '/home/nivg/data/seascape_naive.jpg'
IM_NAME = 'seascape'
pyr = get_pyramid(IM_PATH, None, 0, 15, 0.75, print_=True) 
pyr_harm = get_pyramid(HARM_IM_PATH, None, 0, 15, 0.75, print_=True) 

In [ ]:
# Starry night Harmonization
IM_PATH = '/home/nivg/data/starry_night.png'
HARM_IM_PATH = '/home/nivg/data/starry_night_naive.png'
IM_NAME = 'starry_night'
pyr = get_pyramid(IM_PATH, None, 0, 15, 0.75, print_=True) 
pyr_harm = get_pyramid(HARM_IM_PATH, None, 0, 15, 0.75, print_=True) 

In [ ]:
im = harmonization(pyr, pyr_harm, top_level=0, patch_size=7)
# imwrite('/home/nivg/data/DropGAN/image_harmonization/' + IM_NAME + '_harmonized.png', im.squeeze(0).detach().cpu())

In [ ]:
# Tree Edit
IM_PATH = '/home/nivg/data/stone.png'
EDIT_IM_PATH = '/home/nivg/data/stone_edit.png'
IM_NAME = 'stone'
pyr = get_pyramid(IM_PATH, None, 0, 15, 0.75, print_=True)
pyr_edit = get_pyramid(EDIT_IM_PATH, None, 0, 15, 0.75, print_=True)

In [ ]:
# for i in range(10):
#     im = edit(pyr, pyr_edit, mask_size=44, patch_size=7, top_level_max=9)
#     imwrite('/home/nivg/data/DropGAN/image_editing/' + IM_NAME + '_edited_ver' + str(i) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Stone Edit
IM_PATH = '/home/nivg/data/stone.png'
EDIT_IM_PATH = '/home/nivg/data/stone_edit.png'
IM_NAME = 'stone'
pyr = get_pyramid(IM_PATH, None, 0, 15, 0.75, print_=True) 
pyr_edit = get_pyramid(EDIT_IM_PATH, None, 0, 15, 0.75, print_=True) 

im = edit(pyr, pyr_edit, mask_size=44, patch_size=7, top_level_max=9)
# imwrite('/home/nivg/data/DropGAN/image_editing/' + IM_NAME + '_edited.png', im.squeeze(0).detach().cpu())

In [ ]:
# # Structural analogy
# for im_a, im_b in [['205.jpg', '204.jpg'], ['204.jpg', '205.jpg'], ['208_resized.jpg', '209.jpg'], ['209.jpg', '208_resized.jpg'], 
#                    ['203.png', '202.png'], ['202.png', '203.png'], ['206.jpg', '207.jpg'], ['207.jpg', '206.jpg']]:

#     IM_PATH_A = '/home/nivg/data/' + im_a
#     IM_PATH_B = '/home/nivg/data/' + im_b
# #     im, pyr_a = structural_analogy(IM_PATH_A, IM_PATH_B, print_=True, top_layer=8)
# #     im1 = structural_analogy_refinement(im, pyr_a, patch_size=7)
#     im = 
#     imshow(im)
#     imshow(im1)

In [ ]:
IM_PATH_A = '/home/nivg/data/207.jpg'
IM_PATH_B = '/home/nivg/data/206.jpg'
im = structural_analogy(IM_PATH_A, IM_PATH_B, patch_size=7, top_layer=8, eps=5e-2, print_=False)
structural_analogy_refinement(im)

In [ ]:
SUMMARY_IM_PATH = '/home/nivg/data/farm_house_s.png'
pyr = get_pyramid(SUMMARY_IM_PATH, None, 0, 22, 0.8, print_=False) 
pyr = pyr[2:]
print(pyr[0].shape)
im = summarization(pyr, ratio=(2,1), print_=True, min_size=35, weighted_mean=True)

In [ ]:
pyr1 = get_pyramid('/home/nivg/data/Tapestry_input_1.png', None, 0, 15, 0.8, print_=True) # Summary
pyr2 = get_pyramid('/home/nivg/data/Tapestry_input_2.png', None, 0, 15, 0.8, print_=True) # Summary
pyr3 = get_pyramid('/home/nivg/data/Tapestry_input_3_resized.png', None, 0, 15, 0.8, print_=True) # Summary

Montage([pyr1, pyr2, pyr3], num_steps=9, print_=False, patch_size=7, min_size=25)

In [ ]:
# Conditional Inpainting
pyr = get_pyramid('/home/nivg/data/204.jpg', None, 0, 15,0.75)
mask = torch.ones_like(pyr[0])
mask[:,:,50:75, 20:40] = 0
new_im = pyr[0].clone()
for i in range(3):
    if i == 2:
        new_im[:,i,50:75, 20:40] = 0
    else:
        new_im[:,i,50:75, 20:40] = 1
conditional_inpainting(new_im, mask, patch_size=7, top_layer=5, print_=False)

In [ ]:
# Conditional Inpainting for figure
pyr = get_pyramid('/home/nivg/data/BSDS100/296007.png', None, 0, 15,0.75)
pyr = pyr[2:]
mask = torch.ones_like(pyr[0])
mask[:,:,130:165, 160:190] = 0
new_im = pyr[0].clone()#*mask
imshow(new_im)
for i in range(2):
    if i == 0:
        new_im[:,:,140:165, 160:190] = 0.0
    else:
        for j in range(3):
            if j < 2:
                new_im[:,j,140:165, 160:190] = 0.7
            else:
                new_im[:,j,140:165, 160:190] = 0.3
    conditional_inpainting(new_im, mask, patch_size=7, top_layer=4, print_=False)